In [10]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
from rdkit.Chem import AllChem

import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit
from src.utils.constants import ES_THRESHOLD, METAL_OX_STATES
from tqdm import tqdm 

In [11]:
from src.data.utils import pdb_to_rdkit_mol, mol2_to_rdkit_mol, get_vdw_radius, add_charges_to_molecule, get_node_features, get_edge_features, extract_charges_from_mol2, ionic_interaction

In [12]:
def calculate_gasteiger_charges(mol):
    AllChem.ComputeGasteigerCharges(mol)
    for atom in mol.GetAtoms():
        atom.SetProp('_TriposAtomCharges', str(atom.GetDoubleProp('_GasteigerCharge')))
    return mol


In [13]:
# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1), # expected output:  None
              ("CC(=O)O", 2, 3) # expected output: (0, 0, 0, 0, 0)
              ]

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    mol = calculate_gasteiger_charges(mol)
    molecules.append((mol, a1, a2))

# Run test cases
for mol, atom_1, atom_2 in molecules:
    # Replace this with the actual function call
    result = ionic_interaction(mol, atom_1, atom_2)
    print(f"SMILES: {Chem.MolToSmiles(mol)}, Atoms: ({atom_1}, {atom_2}) -> Result: {result}")


SMILES: C=CCCCCCCCCC, Atoms: (0, 10) -> Result: None
SMILES: C=C, Atoms: (0, 1) -> Result: None
SMILES: CC(=O)O, Atoms: (2, 3) -> Result: (0, 0, 0, 0, 0, 74.1944410471423)


[22:55:36] Molecule does not have explicit Hs. Consider calling AddHs()
[22:55:36] Molecule does not have explicit Hs. Consider calling AddHs()
[22:55:36] Molecule does not have explicit Hs. Consider calling AddHs()


In [14]:
complex_mol = Chem.RemoveHs(mol2_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2", sanitize = False), sanitize = False) #/Users/tsachmackey/Documents/Summer 2024/Research /Batista project/AffinityNet/test_data/pdb/1a0q
ligand_mol = Chem.RemoveHs(mol2_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_ligand.mol2", sanitize = False), sanitize = False) #/Users/tsachmackey/Documents/Summer 2024/Research /Batista project/AffinityNet/test_data/pdb/1a0q
num_atoms_in_protein = len(complex_mol.GetAtoms()) - len(ligand_mol.GetAtoms())
charges = extract_charges_from_mol2("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2")

mol = complex_mol
AssignStereochemistry(mol, force=True, cleanIt=True)
protein_or_ligand_ids = [-1 if atom.GetIdx() < num_atoms_in_protein else 1 for atom in mol.GetAtoms()]

In [15]:
add_charges_to_molecule(mol, charges)
pocket_atom_indices = find_pocket_atoms_RDKit(mol, protein_or_ligand_ids, num_atoms_in_protein) # just use all of the atoms for now 
# pocket_atom_indices = [atom.GetIdx() for atom in mol.GetAtoms()]
edge_features, edge_indices = get_edge_features(mol, pocket_atom_indices, pairwise_function=ionic_interaction)

Zn
Zn
Zn


100%|██████████| 3209/3209 [00:00<00:00, 184127.30it/s]
653it [00:00, 668.93it/s]


In [16]:
# Extract the last column from edge_features
last_column = abs(edge_features[:, -1])

# Sort edge_indices based on the last column
sorted_indices = np.argsort(last_column)
sorted_edge_indices = edge_indices[sorted_indices]
sorted_edge_features = edge_features[sorted_indices]

print("Length of sorted_edge_indices:", len(sorted_edge_indices))
print("Sorted Edge Indices:", sorted_edge_indices[:100])

Length of sorted_edge_indices: 254
Sorted Edge Indices: [[2418 3184]
 [3184 2418]
 [3185  714]
 [ 714 3185]
 [3185 1876]
 [1876 3185]
 [2048 3184]
 [3184 2048]
 [3204 3185]
 [3185 3204]
 [3184 2034]
 [2034 3184]
 [ 723 3185]
 [3185  723]
 [3185 2095]
 [2095 3185]
 [3184 2030]
 [2030 3184]
 [1877 3185]
 [3185 1877]
 [ 703 3185]
 [3185  703]
 [2027 3185]
 [3185 2027]
 [2073 3184]
 [3184 2073]
 [2010 3185]
 [3185 2010]
 [ 715 3185]
 [3185  715]
 [3185 2093]
 [2093 3185]
 [3184 2419]
 [2419 3184]
 [3184 3203]
 [3203 3184]
 [3185 2048]
 [2048 3185]
 [2017 3185]
 [3185 2017]
 [2433 3184]
 [3184 2433]
 [3202 3184]
 [3184 3202]
 [3202 3185]
 [3185 3202]
 [2182 3185]
 [3185 2182]
 [ 711 3185]
 [3185  711]
 [2400 3184]
 [3184 2400]
 [1666 3184]
 [3184 1666]
 [3184 1880]
 [1880 3184]
 [3185 3203]
 [3203 3185]
 [1879 3184]
 [3184 1879]
 [2090 3185]
 [3185 2090]
 [ 736 3185]
 [3185  736]
 [3184  263]
 [ 263 3184]
 [1665 3184]
 [3184 1665]
 [2018 3185]
 [3185 2018]
 [2039 3185]
 [3185 2039]
 [3184 2

In [17]:
print("Sorted Edge Features:", sorted_edge_features)

Sorted Edge Features: [[  0.           0.           0.           0.           0.
  -50.25308099]
 [  0.           0.           0.           0.           0.
  -50.25308099]
 [  0.           0.           0.           0.           0.
   50.30233485]
 ...
 [  0.           0.           0.           0.           0.
  292.41441035]
 [  0.           0.           0.           0.           0.
  334.94795218]
 [  0.           0.           0.           0.           0.
  334.94795218]]


Taking a look at the top interaction ... 

In [18]:
def print_atom_facts(index): 
    atom = mol.GetAtomWithIdx(index)
    print(f"Atom index: {atom.GetIdx()}")
    print(f"Atom symbol: {atom.GetSymbol()}")
    print(f"Atom atomic number: {atom.GetAtomicNum()}")
    print(f"Atom partial charge: {atom.GetProp('_TriposAtomCharges')}")

print_atom_facts(3185)
print_atom_facts(2074)

Atom index: 3185
Atom symbol: Zn
Atom atomic number: 30
Atom partial charge: 2
Atom index: 2074
Atom symbol: O
Atom atomic number: 8
Atom partial charge: -0.27000000000000002


Looking at a top non-metal interaction ...

In [19]:
print_atom_facts(2073)
print_atom_facts(2030)

Atom index: 2073
Atom symbol: C
Atom atomic number: 6
Atom partial charge: 0.35499999999999998
Atom index: 2030
Atom symbol: O
Atom atomic number: 8
Atom partial charge: -0.30199999999999999
